In [ ]:
# Figures for: "Limits to Complex Infinities"
# Generates and saves:
#   1) fig_circle_compactification.png
#   2) fig_spiral_compactification.png
#   3) fig_stereographic_directional.png
#   4) fig_riemann_sphere.png
#   5) fig_wrapping_dense_vs_rational.png
#
# Tip: change "out_ext" to ".pdf" or ".svg" if you want vector outputs.

import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D  # noqa: F401 (enables 3D)

out_dir = "/mnt/data"
out_ext = ".png"  # set to ".pdf" or ".svg" if desired

# --------------------------
# Helpers
# --------------------------
def savefig(path):
    plt.tight_layout()
    plt.savefig(path, dpi=300, bbox_inches="tight")
    plt.show()
    print(f"Saved: {path}")

def unit_circle(ax):
    th = np.linspace(0, 2*np.pi, 400)
    ax.plot(np.cos(th), np.sin(th))
    ax.set_aspect("equal", "box")
    ax.set_xlim([-1.1, 1.1])
    ax.set_ylim([-1.1, 1.1])
    ax.set_xlabel("x")
    ax.set_ylabel("y")
    ax.grid(True, linestyle=":", linewidth=0.5)

def stereographic_map_plane_to_sphere(x, y):
    r2 = x*x + y*y
    X = 2*x / (r2 + 1)
    Y = 2*y / (r2 + 1)
    Z = (r2 - 1) / (r2 + 1)
    return X, Y, Z

# --------------------------
# Figure 1: Circle compactification (tangent half-angle)
# --------------------------
def figure_circle_compactification():
    t = np.linspace(-10, 10, 400)
    x = (1 - t**2) / (1 + t**2)
    y =  2*t       / (1 + t**2)

    fig, ax = plt.subplots(figsize=(5, 5))
    unit_circle(ax)
    ax.plot(x, y, linewidth=1.0)

    # sample markers to show approach to north pole
    t_marks = np.array([-20, -5, -1, 0, 1, 5, 20])
    xm = (1 - t_marks**2) / (1 + t_marks**2)
    ym =  2*t_marks       / (1 + t_marks**2)
    ax.scatter(xm, ym)
    ax.scatter([0], [1])  # north pole marker

    ax.set_title("Circle Compactification (tangent half-angle): $\\mathbb{R}\\cup\\{\\infty\\}\\cong S^1$")
    savefig(f"{out_dir}/fig_circle_compactification{out_ext}")

# --------------------------
# Figure 2: Spiral compactification on the unit disk
# --------------------------
def figure_spiral_compactification():
    x = np.linspace(-30*np.pi, 30*np.pi, 4000)
    r = 1 - np.exp(-np.abs(x))
    X = r * np.cos(x)
    Y = r * np.sin(x)

    fig, ax = plt.subplots(figsize=(5, 5))
    unit_circle(ax)
    ax.plot(X, Y, linewidth=0.8)
    ax.set_title("Spiral Compactification in Unit Disk: boundary circle ≡ one point")
    savefig(f"{out_dir}/fig_spiral_compactification{out_ext}")

# --------------------------
# Figure 3: Stereographic projection with directional approaches to N
# --------------------------
def figure_stereographic_directional():
    u = np.linspace(0, 2*np.pi, 60)
    v = np.linspace(0, np.pi, 30)
    Xs = np.outer(np.cos(u), np.sin(v))
    Ys = np.outer(np.sin(u), np.sin(v))
    Zs = np.outer(np.ones_like(u), np.cos(v))

    fig = plt.figure(figsize=(6, 6))
    ax = fig.add_subplot(111, projection="3d")
    ax.plot_wireframe(Xs, Ys, Zs, linewidth=0.3)

    phis = np.deg2rad([0, 30, 60, 120, 150, 210, 240, 300, 330])
    rs = np.linspace(0.5, 15.0, 40)
    for phi in phis:
        xp = rs*np.cos(phi)
        yp = rs*np.sin(phi)
        Xp, Yp, Zp = stereographic_map_plane_to_sphere(xp, yp)
        ax.plot(Xp, Yp, Zp, linewidth=1.0)

    ax.scatter([0], [0], [1])  # North pole
    ax.set_xlabel("X")
    ax.set_ylabel("Y")
    ax.set_zlabel("Z")
    ax.set_title("Stereographic Projection: directional approaches to $N$ on $S^2$")
    ax.set_box_aspect((1, 1, 1))
    savefig(f"{out_dir}/fig_stereographic_directional{out_ext}")

# --------------------------
# Figure 4: Riemann sphere depiction
# --------------------------
def figure_riemann_sphere():
    fig = plt.figure(figsize=(7, 6))
    ax = fig.add_subplot(111, projection="3d")

    # Sphere surface
    u = np.linspace(0, 2*np.pi, 80)
    v = np.linspace(0, np.pi, 40)
    Xs = np.outer(np.cos(u), np.sin(v))
    Ys = np.outer(np.sin(u), np.sin(v))
    Zs = np.outer(np.ones_like(u), np.cos(v))
    ax.plot_wireframe(Xs, Ys, Zs, linewidth=0.3)

    # Plane grid mapped to sphere
    grid_lim = 2.0
    grid_vals = np.linspace(-grid_lim, grid_lim, 9)
    for gv in grid_vals:
        x = np.full_like(grid_vals, gv)  # verticals
        y = grid_vals
        Xp, Yp, Zp = stereographic_map_plane_to_sphere(x, y)
        ax.plot(Xp, Yp, Zp, linewidth=0.6)

        y = np.full_like(grid_vals, gv)  # horizontals
        x = grid_vals
        Xp, Yp, Zp = stereographic_map_plane_to_sphere(x, y)
        ax.plot(Xp, Yp, Zp, linewidth=0.6)

    # Line y=0 mapped (circle through ∞)
    x_line = np.linspace(-3.0, 3.0, 200)
    y_line = np.zeros_like(x_line)
    Xl, Yl, Zl = stereographic_map_plane_to_sphere(x_line, y_line)
    ax.plot(Xl, Yl, Zl, linewidth=2.0)

    # Circle |z|=R mapped to a circle on the sphere
    R = 1.5
    th = np.linspace(0, 2*np.pi, 300)
    x_c = R*np.cos(th)
    y_c = R*np.sin(th)
    Xc, Yc, Zc = stereographic_map_plane_to_sphere(x_c, y_c)
    ax.plot(Xc, Yc, Zc, linewidth=2.0)

    ax.scatter([0], [0], [1])  # ∞ (north pole)
    ax.set_xlabel("X")
    ax.set_ylabel("Y")
    ax.set_zlabel("Z")
    ax.set_title("Riemann Sphere: plane at equator; lines/circles ↔ circles on $S^2$")
    ax.set_box_aspect((1, 1, 1))
    savefig(f"{out_dir}/fig_riemann_sphere{out_ext}")

# --------------------------
# Figure 5: Periodic wrapping demo — rational vs. irrational
# --------------------------
def figure_wrapping_dense_vs_rational():
    N = 400
    p, q = 1, 12
    alpha_rational = 2*np.pi * (p/q)

    phi = (np.sqrt(5) - 1) / 2  # ~0.618..., irrational
    alpha_irrational = 2*np.pi * phi

    n = np.arange(N)
    z_rat = np.exp(1j * alpha_rational * n)
    z_irr = np.exp(1j * alpha_irrational * n)

    fig, ax = plt.subplots(figsize=(5, 5))
    unit_circle(ax)
    ax.scatter(np.real(z_rat), np.imag(z_rat), s=8, label=f"Rational step p/q={p}/{q}")
    ax.scatter(np.real(z_irr), np.imag(z_irr), s=8, label="Irrational step (golden ratio)")
    ax.legend(loc="upper right", frameon=True)
    ax.set_title("$n\\mapsto e^{i\\alpha n}$ on $S^1$: finite cycle vs. dense coverage")
    savefig(f"{out_dir}/fig_wrapping_dense_vs_rational{out_ext}")

# --------------------------
# Generate all figures
# --------------------------
figure_circle_compactification()
figure_spiral_compactification()
figure_stereographic_directional()
figure_riemann_sphere()
figure_wrapping_dense_vs_rational()

: 

In [ ]:
# Reproducible figures for:
#   1) adaptive_amplitude_n_up_to_10000.png
#   2) sin_heatmap_np.png
#   3) coverage_vs_primebound.png
#
# Requirements: numpy, matplotlib
# Notes:
#  - No seaborn, no custom styles; each figure created separately.
#  - Computations are modest; should run fast up to X ~ 1e4–5e4.

import numpy as np
import matplotlib.pyplot as plt

# -------------------------------
# Prime utilities
# -------------------------------
def primes_upto(N: int) -> np.ndarray:
    """Return all primes <= N via a simple sieve."""
    if N < 2:
        return np.array([], dtype=int)
    sieve = np.ones(N + 1, dtype=bool)
    sieve[:2] = False
    lim = int(N**0.5)
    for p in range(2, lim + 1):
        if sieve[p]:
            sieve[p*p:N+1:p] = False
    return np.nonzero(sieve)[0]

# -------------------------------
# Model B: Adaptive amplitude A(n)
# -------------------------------
def adaptive_amplitude_array(max_n: int) -> np.ndarray:
    """
    Compute A(n) = prod_{p <= sqrt(n)} |sin(2π n / p)| for n = 2..max_n.
    Uses a log-sum trick to avoid underflow; we exponentiate at the end,
    but keep exact zeros when any factor is (numerically) ~0.
    """
    primes = primes_upto(int(np.sqrt(max_n)) + 1)  # sufficient set for all n <= max_n
    n_vals = np.arange(2, max_n + 1, dtype=float)

    # logA initialized to 0; we accumulate logs for each prime p
    logA = np.zeros_like(n_vals)
    is_zero = np.zeros_like(n_vals, dtype=bool)

    # small threshold for treating sin(...) as 0 (avoid float noise)
    eps = 1e-15

    for p in primes:
        # mask: only apply factor for n with p <= sqrt(n)
        mask = (p <= np.sqrt(n_vals))
        if not np.any(mask):
            continue

        # |sin(2π n / p)|
        s = np.abs(np.sin(2.0 * np.pi * n_vals[mask] / p))

        # exact zero detection (within machine tol)
        zero_mask = (s < eps)
        if np.any(zero_mask):
            is_zero[np.where(mask)[0][zero_mask]] = True

        # accumulate logs but avoid log(0) for zeros (we already marked them)
        safe = ~zero_mask
        if np.any(safe):
            logA[np.where(mask)[0][safe]] += np.log(s[safe])

    # Exponentiate; keep zeros where flagged
    A = np.exp(logA)
    A[is_zero] = 0.0
    return A

# -------------------------------
# Figure 1: Adaptive amplitude up to 10^4
# -------------------------------
def fig_adaptive_amplitude(max_n: int = 10_000, filename: str = "adaptive_amplitude_n_up_to_10000.png"):
    A = adaptive_amplitude_array(max_n)
    n = np.arange(2, max_n + 1)

    plt.figure(figsize=(10, 4))
    plt.plot(n, A, ".", markersize=2)
    plt.title(r"Adaptive amplitude $A(n)$ up to $10^4$")
    plt.xlabel("n")
    plt.ylabel("A(n)")
    plt.tight_layout()
    plt.savefig(filename, dpi=300, bbox_inches="tight")
    plt.show()
    print(f"Saved: {filename}")

# -------------------------------
# Figure 2: Heatmap of |sin(2π n / p)|
# -------------------------------
def fig_sin_heatmap(N: int = 500, Pmax: int = 97, filename: str = "sin_heatmap_np.png"):
    n_vals = np.arange(1, N + 1, dtype=float)
    p_vals = primes_upto(Pmax)
    M = np.zeros((len(p_vals), N), dtype=float)

    # Fill heatmap: rows over primes p, cols over n
    for i, p in enumerate(p_vals):
        M[i, :] = np.abs(np.sin(2.0 * np.pi * n_vals / p))

    plt.figure(figsize=(10, 6))
    # extent so y-axis shows p values top->bottom
    plt.imshow(M, aspect="auto", origin="upper",
               extent=[1, N, p_vals[0], p_vals[-1]])
    cbar = plt.colorbar()
    cbar.set_label(r"$|\sin(2\pi n/p)|$")
    plt.ylabel("prime p")
    plt.xlabel("n")
    plt.title(r"Heatmap of $|\sin(2\pi n/p)|$  (rows: primes $p\leq 97$, cols: $1\leq n\leq 500$)")
    plt.tight_layout()
    plt.savefig(filename, dpi=300, bbox_inches="tight")
    plt.show()
    print(f"Saved: {filename}")

# -------------------------------
# Figure 3: Coverage vs prime bound
# -------------------------------
def coverage_fraction_upto_X(X: int, P: int, primes_all: np.ndarray) -> float:
    """
    Mark multiples of primes up to P among 1..X (inclusive) and
    return the coverage fraction. (Marks 0/1 vector; 0 and 1 are ignored.)
    """
    marked = np.zeros(X + 1, dtype=bool)
    for p in primes_all:
        if p > P:
            break
        marked[p::p] = True
    # exclude 0; include 1..X
    return marked[1:].mean()

def fig_coverage_vs_primebound(X: int = 5000, filename: str = "coverage_vs_primebound.png"):
    primes_all = primes_upto(X)
    P_vals = primes_all  # evaluate coverage at each prime cutoff
    cov = np.array([coverage_fraction_upto_X(X, P, primes_all) for P in P_vals])

    plt.figure(figsize=(8, 5))
    plt.plot(P_vals, cov, linewidth=1.5)
    plt.xlabel("Prime bound P")
    plt.ylabel("Coverage fraction ≤ X")
    plt.title(f"Coverage ratio vs prime bound  (X={X})")
    plt.tight_layout()
    plt.savefig(filename, dpi=300, bbox_inches="tight")
    plt.show()
    print(f"Saved: {filename}")

# -------------------------------
# Run all figures
# -------------------------------
fig_adaptive_amplitude()                 # -> adaptive_amplitude_n_up_to_10000.png
fig_sin_heatmap()                        # -> sin_heatmap_np.png
fig_coverage_vs_primebound()             # -> coverage_vs_primebound.png


: 